In [1]:
import pandas as pd
import numpy as np
from numpy import unique
path = 'C:/Users/paylo/Desktop/projeto1/prouni_2005_2019.csv'
df = pd.read_csv(path,engine = 'python', encoding='utf-8')

In [2]:
df = df.drop('CPF_BENEFICIARIO_BOLSA',axis = 1) # dropamos as colunas que nao agrega em nada e ira deixar mais rapido o script
df = df.drop('DT_NASCIMENTO_BENEFICIARIO',axis = 1)
df['idade'] = df['idade'].astype(int)# transformar que estava em string
df = df.loc[(df['idade'] >= 18)] # Delimitador de idade para o estudo
df = df.loc[(df['idade'] <= 30)]
df = df.dropna()

In [3]:
# Coleçao de funcões para limpeza dos nomes de curso, muitos vieram do csv com problemas em caracteres especiais em portugues
# como Ç,ão,á,é etc.



def troca(x,a,lista):  # funçao geral limpeza de nomes com caracteres expecificos, limpando tudo a direita desses caracterees 
    if x in lista[a]:
        nome = lista[a]
        nome_reduzido = nome.split(x, 1)[0]
        lista[a] = nome_reduzido
    return lista

def troca_AA(a,lista): # notei que sempre que aparecia AA no nome era no lugar de Çã, entao essa funçao troca o AA seguido de o
    if 'AA' in lista[a]:# para cao
        pos_a = lista[a].index('AA')
        indexes = []
        count = 0
        for c in lista[a]:
            if c == 'o':
                indexes.append(count)
            count = count + 1 
        for d in range(len(indexes)):
            pos_o = indexes[d] 
            expected_0 = pos_a + 2
            if expected_0  == pos_o:
                nome = lista[a][0:pos_a] + "ca" + lista[a][pos_o:]
                lista[a] = nome
    return lista


def troca_string(string,a,lista): # tira os espaços no meio de palavras
    if string in lista[a].lower():
        pos_a = lista[a].lower().index(string)
        nome = lista[a][0:pos_a]
        nome_reduzido = nome.split(' ', 1)[0]
        lista[a] = nome_reduzido
    return lista


def troca_A3(a,lista): # novamente notei que onde aparecia a3 deveria ser o
    if 'A3' in lista[a]:
        pos_a = lista[a].index('A3')
        nome = lista[a][0:pos_a] + "o" + lista[a][pos_a+2:]
        lista[a] = nome
    return lista
            
            
            

In [9]:
lista =[]
lista = df['NOME_CURSO_BOLSA'].to_list()
for a in range(len(lista)):
    lista[a] = lista[a].lstrip(' ')
caracteres_ind = ['-',':','(','*']
for a in range(len(lista)):
    lista = troca_AA(a,lista)
    lista = troca_AA(a,lista)
    lista = troca_AA(a,lista)
    lista = troca_A3(a,lista)
    for b in range(len(caracteres_ind)):
        troca(caracteres_ind[b],a,lista)
        
    lista = troca_string('ead',a,lista)
    lista = troca_string('distancia',a,lista)

    
df = df.assign(Curso = lista )

In [8]:

lista2 = df['CODIGO_EMEC_IES_BOLSA'].to_list()
lista2 = np.array(lista2)
lista2 = unique(lista2)

df2 = df.copy()

unicos = list(set(df2['NOME_CURSO_BOLSA'].to_list()))

for a in range(len(lista2)): # condensa os diferentes nomes de curso apos a limpeza com base no codigo do curso.
    numero = lista2[a]
    df2.loc[:, 'CODIGO_EMEC_IES_BOLSA'] = numero
    lista = df2['NOME_CURSO_BOLSA'].to_list()
    lista3 = []
    for b in range(len(lista)):
        lista3.append(sum(1 for c in lista[b] if c.isupper()))
    idx_min = lista3.index(min(lista3))
    nome_min = lista[idx_min]            
    df.loc[df['CODIGO_EMEC_IES_BOLSA']==numero,'NOME_CURSO_BOLSA']= nome_min




In [7]:
df.to_csv('C:/Users/paylo\Desktop/projeto1/prouni_2005_2019_limpo.csv') # salva o dataframe
